In [16]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from copy import deepcopy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt
import cv2

In [3]:
# 需要修改成的数据大小
dsize = 128
# 最大学习率(优化器)
max_lr = 0.012 
# 正则项权值的衰减(优化器)
weight_decay = 1e-4 
# 一般0.9 (优化器)
momentum = 0.9 
# 最小学习率(退火学习)
min_lr = 0.001
# 设置GPU运行
device = torch.device('cuda')

In [4]:
src = './competition_data'
save_weight_path = src + '/weight'
train_image_dir = src + '/train/images'
train_mask_dir = src + '/train/masks'
test_image_dir = src + '/test/images'

depths = pd.read_csv(src + '/depths.csv')
depths.sort_values('z', inplace=True)
depths.drop('z', axis=1, inplace=True)
depths['fold'] = (list(range(0,5)) * depths.shape[0])[:depths.shape[0]]

In [5]:
train_df = pd.read_csv(src + '/train.csv')
train_df = train_df.merge(depths)
dist = []

for id in train_df.id.values:
  # f的使用就是将大括号内的变量转为字符
  img = plt.imread(train_image_dir+f'/{id}.png')
  dist.append(np.unique(img).shape[0])
train_df['unique_pixels'] = dist

# 图片id
all_id = train_df['id'].values
# 数据分为5份
fold = []
for i in range(5):
  fold.append(train_df.loc[train_df['fold']==i, 'id'].values)

In [6]:
# 找到两数组内的不同值
# 数值分成了五份，每一份都能拿来做测试集
# 而其他的就是训练集

train_id = np.setdiff1d(all_id, fold[0])
val_id = fold[0]

In [7]:
# 根据id获取图片
def get_train_images(ids):
  images = []
  masks = []
  for id in ids:
    image = plt.imread(train_image_dir+'/'+id+'.png')[0] / 255
    mask = plt.imread(train_mask_dir+'/'+id+'.png')[0] / 255
    masks.append(mask)
    images.append(image)
  return images,masks
# 获取训练集和验证集
train_images,train_masks = get_train_images(train_id)
val_images,val_masks = get_train_images(val_id)



## 创建数据集类型
数据集类型有三个常用魔法方法
1. 初始化（获取参数）
2. 获取数据（数据处理，返回数据）
3. 获取数据集长度（返回数据集长度）

In [8]:
# 训练数据集
class TensorDataset(Dataset):
  def __init__(self, data, target):
    self.data = data
    self.target = target

  def __getitem__(self, index):
    # 改变尺寸，并且变为张量
    resolved_data = torch.Tensor(
      cv2.resize(self.data[index], dsize=(dsize,dsize))
    ).reshape(1,dsize,dsize)
    # 改变尺寸，并且变为张量
    resolved_target = torch.Tensor(
      cv2.resize(self.target[index], dsize=(dsize,dsize))
    ).reshape(1,dsize,dsize)
    # 返回
    # (1,128,128),(1,128,128)
    return resolved_data,resolved_target

  def __len__(self):
    return len(self.data)

## 创建数据集对象

In [9]:
train_data_set = TensorDataset(train_images,train_masks)
val_data_set = TensorDataset(val_images,val_masks)

# (18,1,128,128)
train_loader = DataLoader(
  train_data_set,
  shuffle=True,
  batch_size=18
) 
# (18,1,128,128)
val_loader = DataLoader(
  val_data_set,
  shuffle=False,
  batch_size=18
) 

## 我的模型

In [10]:
class Decoder(nn.Module):
  def __init__(self,in_size,out_size) -> None:
    super().__init__()
    self.layer1 = nn.ConvTranspose2d(in_channels=in_size,out_channels=out_size,kernel_size=2,stride=2,padding=0)
    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=in_size,out_channels=out_size,stride=1,padding=1,kernel_size=3),
      nn.ReLU()
    )
  def forward(self,x1,x2):
    y1 = self.layer1(x1)
    tem = torch.cat((y1,x2),dim=1)
    y2 = self.layer2(tem)
    return y2

class SimpleUNet(nn.Module):
  def __init__(self) -> None:
    super().__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(in_channels=1,out_channels=4,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=4),
      nn.ReLU()
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(in_channels=4,out_channels=8,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=8),
      nn.ReLU()
    )
    self.layer3 = nn.Sequential(
      nn.Conv2d(in_channels=8,out_channels=16,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=16),
      nn.ReLU()
    )
    self.layer4 = nn.Sequential(
      nn.Conv2d(in_channels=16,out_channels=32,stride=2,padding=1,kernel_size=3),
      nn.BatchNorm2d(num_features=32),
      nn.ReLU()
    )
    self.decoder1 = Decoder(32,16)
    self.decoder2 = Decoder(16,8)
    self.decoder3 = Decoder(8,4)
    self.last = nn.Sequential(
      nn.ConvTranspose2d(in_channels=4,out_channels=1,kernel_size=2,stride=2),
      nn.Conv2d(in_channels=1,kernel_size=5,out_channels=1,padding=4),
      nn.Conv2d(in_channels=1,kernel_size=5,out_channels=1),
      nn.ReLU()
    )
    


  def forward(self,input):
    x1 = self.layer1(input) # torch.Size([18, 4, 64, 64])
    x2 = self.layer2(x1) # torch.Size([18, 8, 32, 32])
    x3 = self.layer3(x2) # torch.Size([18, 16, 16, 16])
    x4 = self.layer4(x3) # torch.Size([18, 32, 8, 8])

    y1 = self.decoder1(x4,x3) # torch.Size([18, 16, 16, 16])
    y2 = self.decoder2(y1,x2)# torch.Size([18, 8, 32, 32])
    y3 = self.decoder3(y2,x1) # torch.Size([18, 4, 64, 64])
    output = self.last(y3) # torch.Size([18, 1, 128, 128])

    return output


## 计算误差

### 优化器
### torch.optim.SGD
|参数|类型|描述|
|-|-|-|
|params|parameters|优化对象的参数|
|lr|float|学习率|
|momentum|float|一般0.9|
|weight_decay|float|正则项权值的衰减系数|

### 退火学习
### torch.optim.lr_scheduler.CosineAnnealingLR
|参数|类型|描述|
|-|-|-|
|optimizer|optimizer|优化器|
|T_max|int|学习率下降到最小值所用周期|
|eta_min|float|学习率最小值|

In [11]:
data_size = len(train_loader)
running_loss = 0.0

salt = SimpleUNet()
salt.to(device=device)
# 优化器
optimizer = torch.optim.SGD(
  params = salt.parameters(), # 优化对象
  lr=max_lr, # 学习率
  momentum=momentum, # 一般0.9
  weight_decay=weight_decay# 正则项权值的衰减
)

    

## 退火学习

In [12]:
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
  optimizer=optimizer,# 优化器
  T_max=300,# 半个余弦
  eta_min=min_lr# 最小学习率
)

## 进行一次训练

In [13]:
# 需要输入数据集，模型
# 返回平均损失

def train(loader_data,model):
  running_loss = 0
  model.train()
  for input,mask in loader_data:
    input, mask = input.to(device), mask.to(device)
    
    optimizer.zero_grad()# 梯度初始化为零
    # 使用with，会自动关闭梯度计算
    # 设置梯度可算
    with torch.set_grad_enabled(True):
      logit = model(input)# 进行一次计算
      loss = nn.BCEWithLogitsLoss()(logit.squeeze(),mask.squeeze())# 计算误差
      loss.backward()# 反馈
      optimizer.step()# 进行一次参数更新
    running_loss += loss.item()*input.size()[0]# 累计平均误差
  epoch_loss = running_loss / len(loader_data)# 计算平均误差
  return epoch_loss

## 进行一次测试

In [61]:
def test(loader_test,model):
  running_loss = 0.0
  data_size = len(loader_test)
  # 测试
  model.eval()
  for input, mask in loader_test:
    input, mask = input.to(device), mask.to(device)
    with torch.set_grad_enabled(False):
      output = model(input)
      loss = nn.BCEWithLogitsLoss()(output.squeeze(), mask.squeeze())
    running_loss += loss.item() * input.size(0)
  return running_loss/data_size